In [2]:
%pip install gradio
%pip install toml

In [1]:
import toml, os, json, re, sys, subprocess, chardet

In [22]:
# load config
config_path = 'config.toml'
config = toml.load(config_path)

In [3]:
anime_folder = config['anime']['anime_folder']
anime_name = config['anime']['anime_name']
# list all mkv files and sort
episodes = sorted([f for f in os.listdir(anime_folder) if f.endswith('.mkv')])
# show mkv subtitles infos
demo_episode = episodes[0]


In [108]:
def path_util(filename, abbrs, ext=None):
    if isinstance(abbrs, str):
        abbrs = [abbrs]
    
    output_path = config['path']['output_folder']
    path = os.path.join(output_path, *abbrs, filename)
    folder = os.path.dirname(path)
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    if ext:
        path = os.path.splitext(path)[0] + ext
    
    return path

path_util('test.mkv', ['a', 'b'], '.srt')

'./output\\a\\b\\test.srt'

In [14]:
def exec_cmd(cmd, print_result=False):
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    encoding = chardet.detect(out)
    if encoding['encoding'] is None:
        result1 = ""
    else:
        result1 = out.decode(encoding['encoding'])
    if print:
        print(result1)
        
    encoding = chardet.detect(err)
    if encoding['encoding'] is None:
        result2 = ""
    else:
        result2 = err.decode(encoding['encoding'])
    if print:
        print(result2)
    return result1, result2
exec_cmd('dir')

 驱动器 C 中的卷没有标签。
 卷的序列号是 103A-F00F

 c:\Users\River\Models\AniVoiceDatasetPreparer 的目录

2024/08/03  22:23    <DIR>          .
2024/07/29  19:23    <DIR>          ..
2024/08/03  22:16             3,234 app.py
2024/08/03  19:26               393 config.toml
2024/08/03  20:41               452 episode.py
2024/08/03  19:49    <DIR>          flagged
2024/08/03  22:39           186,986 main.ipynb
2024/08/03  19:40    <DIR>          output
2024/08/03  22:23    <DIR>          output_segments
2024/08/03  18:29                 0 requirements.txt
2024/08/03  22:33             1,791 subtitle.py
2024/08/03  18:21    <DIR>          Tools
2024/08/03  22:36               869 utils.py
2024/08/03  22:37    <DIR>          __pycache__
               7 个文件        193,725 字节
               7 个目录 438,101,635,072 可用字节




(' 驱动器 C 中的卷没有标签。\r\n 卷的序列号是 103A-F00F\r\n\r\n c:\\Users\\River\\Models\\AniVoiceDatasetPreparer 的目录\r\n\r\n2024/08/03  22:23    <DIR>          .\r\n2024/07/29  19:23    <DIR>          ..\r\n2024/08/03  22:16             3,234 app.py\r\n2024/08/03  19:26               393 config.toml\r\n2024/08/03  20:41               452 episode.py\r\n2024/08/03  19:49    <DIR>          flagged\r\n2024/08/03  22:39           186,986 main.ipynb\r\n2024/08/03  19:40    <DIR>          output\r\n2024/08/03  22:23    <DIR>          output_segments\r\n2024/08/03  18:29                 0 requirements.txt\r\n2024/08/03  22:33             1,791 subtitle.py\r\n2024/08/03  18:21    <DIR>          Tools\r\n2024/08/03  22:36               869 utils.py\r\n2024/08/03  22:37    <DIR>          __pycache__\r\n               7 个文件        193,725 字节\r\n               7 个目录 438,101,635,072 可用字节\r\n',
 '')

In [105]:
def extract_mkv_subtitle_info(file):
    config = toml.load('config.toml')
    mkvinfo = config['tool']['mkvinfo']
    os_cmd = f"{mkvinfo} \"{anime_folder}/{file}\""
    # os_cmd = f"{mkvextract}"
    # print(os_cmd)
    result = exec_cmd(os_cmd)
    # 找到result中包含'轨道编号'的行
    # print(result)
    
    lines = result.split('\n')
    tracks = []
    for lid, line in enumerate(lines):
        if '轨道编号' in line and '字幕' in lines[lid+2]:
            print(line)
            print(lines[lid+1])
            print(lines[lid+2])
            print(lines[lid+4])
            print(lines[lid+5])
            print(lines[lid+7])
            print(lines[lid+8])
            print("===============")
    
extract_mkv_subtitle_info(demo_episode)

|  + 轨道编号: 4 (mkvmerge & mkvextract 的轨道 ID: 3)
|  + 轨道 UID: 14831223280445349528
|  + 轨道类型: 字幕
|  + 语言: chi
|  + 编码格式 ID: S_TEXT/ASS
|  + 语言 (IETF BCP 47): zh-Hans
|  + 名称: JPSC
|  + 轨道编号: 5 (mkvmerge & mkvextract 的轨道 ID: 4)
|  + 轨道 UID: 15973822311672155975
|  + 轨道类型: 字幕
|  + ｢紧缩｣ 标记: 0
|  + 语言: chi
|  + 编解码器私有数据: 大小 2520
|  + 语言 (IETF BCP 47): zh-Hant


In [115]:
def extract_subtitle(file, track_id):
    config = toml.load('config.toml')
    mkvextract = config['tool']['mkvextract']
    path = path_util(file, [anime_name, 'subtitles'], '.srt')
    os_cmd = f"{mkvextract} tracks \"{anime_folder}/{file}\" {track_id}:{path}"
    result = exec_cmd(os_cmd)
    print(result)

extract_subtitle(demo_episode, 3)

Tools\mkvtoolnix\mkvextract.exe tracks "\\RiverNAS\RiverNAS\Bangumi\[Nekomoe kissaten&LoliHouse] Dungeon Meshi [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2]/[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].mkv" 3:test.srt
正在将 CodecID 为 S_TEXT/ASS 的轨道 3 提取到文件｢test.srt｣。容器格式: SSA/ASS text subtitles
进度: 100%



In [4]:
demo_srt = "C:\\Users\\River\\Models\\AniVoiceDatasetPreparer\\output\\Dungeon Meshi\\subtitles\\[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].ass"

In [5]:
demo_episode

'[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].mkv'

In [43]:
import ffmpeg
import pysrt

def extract_audio_segments(mkv_file, srt_file, output_dir):
    # 读取SRT文件
    subs = pysrt.open(srt_file, encoding='utf-8', error_handling=pysrt.ERROR_LOG)
    content = subs.data
    print(content)
    print(subs)
    for i, sub in enumerate(subs):
        # 获取字幕的开始时间和结束时间
        start_time = sub.start.to_time()
        end_time = sub.end.to_time()
        
        # 转换时间格式为ffmpeg可接受的格式
        start_time_str = f'{start_time.hour:02}:{start_time.minute:02}:{start_time.second:02}.{start_time.microsecond//1000:03}'
        end_time_str = f'{end_time.hour:02}:{end_time.minute:02}:{end_time.second:02}.{end_time.microsecond//1000:03}'
        
        print(start_time_str, end_time_str)
        # # 计算剪切的持续时间
        # duration_str = f'{(end_time - start_time).total_seconds():.3f}'
        
        # # 输出文件名
        # output_file = f"{output_dir}/segment_{i+1:03}.mp3"
        
        # # 使用ffmpeg提取音频段
        # (
        #     ffmpeg
        #     .input(mkv_file, ss=start_time_str, t=duration_str)
        #     .output(output_file)
        #     .run(overwrite_output=True)
        # )
        # print(f"Extracted segment {i+1} to {output_file}")

# 示例用法
mkv_file = demo_episode
srt_file = demo_srt
output_dir = 'output_segments'
extract_audio_segments(mkv_file, srt_file, output_dir)


PySRT-InvalidItem(line 755): 


TypeError: write() argument must be str, not <class 'bytes'>

In [7]:
demo_srt

'C:\\Users\\River\\Models\\AniVoiceDatasetPreparer\\output\\Dungeon Meshi\\subtitles\\[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].ass'

In [25]:
import ffmpeg
import pyass
import os
from datetime import timedelta

# def ass_time_to_timedelta(ass_time):
#     parts = ass_time.split(':')
#     hours = int(parts[0])
#     minutes = int(parts[1])
#     seconds, milliseconds = map(int, parts[2].split('.'))
#     return timedelta(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)

def extract_audio_segments(mkv_file, ass_file, output_dir):
    # 读取ASS文件
    with open(ass_file, 'r', encoding='utf-8') as f:
        doc = pyass.load(f)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for i, event in enumerate(doc.events):
        # 获取字幕的开始时间和结束时间
        
        start_time = event.start
        end_time = event.end
        text = event.text
        print(text)
        
        # 转换时间格式为ffmpeg可接受的格式
        start_time_str = str(start_time)
        duration = end_time - start_time
        duration_str = str(duration)
        
        print(start_time, end_time)
        
        # 输出文件名
        output_file = f"{output_dir}/segment_{i+1:03}.mp3"
        
        print(mkv_file, output_file)
        
        # 使用ffmpeg提取音频段
        ffmpeg = config['tool']['ffmpeg']
        os_cmd = f"\"{ffmpeg}\" -i \"{mkv_file}\" -ss {start_time_str} -t {duration_str} \"{output_file}\""
        print(os_cmd)
        exec_cmd(os_cmd, print_result=True)
        
        print(f"Extracted segment {i+1} to {output_file}")
        
        break

# 示例用法
mkv_file = os.path.join(anime_folder, demo_episode)
ass_file = demo_srt
output_dir = 'output_segments'
extract_audio_segments(mkv_file, ass_file, output_dir)


Staff
0:23:12.11 0:23:14.11
\\RiverNAS\RiverNAS\Bangumi\[Nekomoe kissaten&LoliHouse] Dungeon Meshi [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2]\[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].mkv output_segments/segment_001.mp3
"Tools\QuickCut v1.8.0 (已包含FFmpeg)\QuickCut\ffmpeg.exe" -i "\\RiverNAS\RiverNAS\Bangumi\[Nekomoe kissaten&LoliHouse] Dungeon Meshi [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2]\[Nekomoe kissaten&LoliHouse] Dungeon Meshi - 01 [WebRip 1080p HEVC-10bit AAC EAC3 ASSx2].mkv" -ss 0:23:12.11 -t 0:00:02.00 "output_segments/segment_001.mp3"

ffmpeg version 4.4.1-full_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-shared --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable

In [16]:
import ffmpeg

In [ ]:
import ffmpeg.audio


ffmpeg.video.split